In [7]:
import numpy as np
import random
import math

In [43]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
import time
py.sign_in('dabelloa', 'mOxK8BVt2FbvZfiGmK6V')

In [61]:
class individuo:
    x=[];
    f=0;
    sigma=0;
class Individuo:
    x=[];
    f=0;
    sigma=0;
class fSeno:
    '''suma del seno de todas las dimensiones'''
    def fitness(self,x):
        suma = 0
        for i in range(0,len(x)):
            suma += math.sin(x[i])
        return suma
class fSenoSharing:
    bits=16
    limsup=10*math.pi
    liminf=0
    def crecimiento(self, codGenetico):
        dimensiones = int(len(codGenetico)/self.bits)
        x=[]
        for d in range(0,dimensiones):
            xi = 0
            for i in range(0,self.bits):
                xi += (codGenetico[i+self.bits*d]*2**i)
            x.append(xi*(self.limsup-self.liminf)/(2**16) + self.liminf)
        return x
    def f(self,individuo,p):
        x = self.crecimiento(individuo.x)
        suma = 0
        for i in range(0,len(x)):
            suma += math.sin(x[i])
        vecinos = 1
        for i in range(0,len(p)):
            if(self.distancia(x,self.crecimiento(p[i].x)) <= individuo.sigma):
                vecinos += 1
        return suma/vecinos;
    def distancia(self,x1,x2):
        suma = 0
        for i in range(0,len(x1)):
            suma += (x1[i]-x2[i])**2
        return math.sqrt(suma)
    def distanciaBinarios(self,x1,x2):
        suma = 0
        for i in range(0,len(x1)):
            suma += abs(x1[i]-x2[i])
        return suma

In [36]:
class M1:
    def fitness(self,x):
        suma = 0
        for i in range(0, len(x)):
            suma += self.f(x[i])
        return suma
    def f(self,xint):
        return math.pow(math.sin(5.0*math.pi*xint), 6.0 )
class M2:
    def fitness(self,x):
        suma = 0
        for i in range(0, len(x)):
            suma += self.f(x[i])
        return suma
    def f(self,xint):
        m1 = M1()
        y = (xint-0.1)/0.8;
        return math.exp(-2.0*0.69314718*y*y)*m1.f(xint)
class M3:
    def fitness(self,x):
        suma = 0
        for i in range(0, len(x)):
            suma += self.f(x[i])
        return suma
    def f(self,xint):
        m1 = M1()
        xlocal = math.pow( xint, 0.75 ) - 0.05;
        return m1.f(xlocal)
class M4:
    def fitness(self,x):
        suma = 0
        for i in range(0, len(x)):
            suma += self.f(x[i])
        return suma
    def f(self,xint):
        m3 = M3()
        v = m3.f(xint);
        y = (xint-0.08)/0.854;
        return math.exp(-2.0*0.69314718*y*y)*v

7.30537793987846e-10

In [7]:
class deterministic_crowding:
    def procedimietoGeneral(self,p,f):
        nuevaGeneracion =[]
        for j in range (0,int(len(p)/2)):
            padres =[]
            padres.append(p[int(random.uniform(0,len(p)))])
            padres.append(p[int(random.uniform(0,len(p)))])
            hijos = self.cruce(padres, f)
            seleccionados = self.reemplazo(padres, hijos)
            for i in range(0, len(seleccionados)):
                nuevaGeneracion.append(seleccionados[i])
        return nuevaGeneracion
    def cruce(self,padres, f):
        hijos=[]
        for h in range(0,2):
            x=[]
            for i in range(0,len(padres[0].x)):
                alpha1=random.random()
                alpha2=random.random()
                alpha1=alpha1/(alpha1+alpha2)
                alpha2=alpha2/(alpha1+alpha2)
                hijo1= individuo()
                x.append(alpha1*padres[0].x[i] + alpha2*padres[1].x[i])
                hijo1.x = x
                hijo1.fitness=f.fitness(hijo1.x);
                hijos.append(hijo1);
        return hijos
    def reemplazo(self,padres, hijos):
        s1 = self.distancia(padres[0],hijos[0]) + self.distancia(padres[1],hijos[1])
        s2 = self.distancia(padres[0],hijos[1]) + self.distancia(padres[1],hijos[0])
        seleccionados=[]
        if s1 < s2:
            if(padres[0].fitness > hijos[0].fitness):
                seleccionados.append(padres[0])
            else:
                seleccionados.append(hijos[0])
            if(padres[1].fitness > hijos[1].fitness):
                seleccionados.append(padres[1])
            else:
                seleccionados.append(hijos[1])
        else:
            if(padres[0].fitness > hijos[1].fitness):
                seleccionados.append(padres[0])
            else:
                seleccionados.append(hijos[1])
            if(padres[1].fitness > hijos[0].fitness):
                seleccionados.append(padres[1])
            else:
                seleccionados.append(hijos[0])
        return seleccionados
    def distancia(self,padre, hijo):
        dist = 0
        for i in range(0,len(padre.x)):
            dist += padre.x[i]
        return dist
    

In [46]:
class AlgoritmoGenetico():
    def iniciarPoblacion(self, n, bits, funcion, sigma):
        poblacion = []
        for i in range(0,n):
            individuo = Individuo()
            individuo.x = np.random.choice([0, 1], size=(bits)).tolist()
            individuo.sigma = sigma
            poblacion.append(individuo)
        for i in range(0,len(poblacion)):
            temp =0
            temp =funcion.f(poblacion[i], poblacion)
            poblacion[i].f = temp
        return poblacion
    def selectorRuleta(self, individuos):
        boletas = []
        conjunto = []
        padres = []
        sumaFitness = 0
        for individuo in individuos:
            sumaFitness += individuo.f
        for individuo in individuos:
            boletas.append(int(len(individuos)*10*individuo.f/sumaFitness)+1)
        for i in range(0,len(individuos)):
            for j in range(0,boletas[i]):
                conjunto.append(i)
        np.random.shuffle(conjunto)
        for i in range(0,len(individuos)):
            padres.append(individuos[conjunto[int(np.random.uniform(0,len(conjunto)))]])
        return padres
    def selectorTorneo(self, individuos):
        padres = []
        for i in range(0,len(individuos)):
            a = np.arange(0,len(individuos)).tolist()
            posCompetidores = np.random.choice(a, size=(4)).tolist()
            posMejor = posCompetidores[0]
            for i in posCompetidores:
                if individuos[i].f > individuos[posMejor].f:
                    posMejor = i
            padres.append(individuos[posMejor])
        return padres
    def selectorUniversalEstocastico(self, individuos):
        individuos = sorted(individuos, key=lambda individuo: individuo.f)
        k = len(individuos)
        sumaF = 0
        for i in individuos:
            sumaF += i.f
        d = sumaF/k
        pi = np.random.uniform(0,d)
        padres = []
        i = 0
        for p in range(0,k):
            while (i < k-1) and (individuos[i].f < pi + p*d):
                i += 1
            padres.append(individuos[i])
        return padres
    def selectorElitista(self, individuos):
        individuos = sorted(individuos, key=lambda individuo: individuo.f, reverse = True)
        elite = int(len(individuos)*0.1)
        claseMedia = int(len(individuos)*0.8)
        padresElite = []
        padresClaseMedia = []
        for i in range(0,int(len(individuos)/2)):
            padresElite.append(individuos[int(np.random.uniform(0,elite))])
            padresClaseMedia.append(individuos[int(np.random.uniform(elite,claseMedia))])
        diferencia = len(individuos) - (len(padresElite) + len(padresClaseMedia))
        while diferencia > 0:
            padresElite.append(individuos[int(np.random.uniform(0,elite))])
            diferencia -= 1
        return padresElite + padresClaseMedia
    def cruceYMutacionGaussianaPoblacion(self, padres, probCruce, mediaMutacion, sigmaMutacion, funcion,sigma):
        a = np.arange(0,len(padres))
        np.random.shuffle(a)
        a = a.reshape(int(len(padres)/2),2).tolist()
        hijos = []
        for pospareja in a:
            if random.random() < probCruce:
                codGenPareja = [padres[pospareja[0]].x, padres[pospareja[1]].x]
                temp = self.cruce(codGenPareja)
                for i in range(0,len(temp)):
                    hijo = Individuo()
                    hijo.x = self.mutacionGaussiana(temp[i], mediaMutacion, sigmaMutacion)
                    hijo.sigma = sigma
                    hijos.append(hijo)
                for i in range(0,len(hijos)):
                    hijos[i].f = funcion.f(hijos[i], poblacion)
            else:
                hijos += [padres[pospareja[0]], padres[pospareja[1]]]
        return hijos
    def cruce(self, codGenPareja):
        punto = int(np.random.uniform(1,len(codGenPareja[0])))
        hijosPareja = []
        hijosPareja.append(codGenPareja[0][:punto]+codGenPareja[1][punto:])
        hijosPareja.append(codGenPareja[1][:punto]+codGenPareja[0][punto:])
        return hijosPareja
    def mutacionGaussiana(self, codGen, media, sigma):
        cantidad = int(random.gauss(media,sigma))
        cantidad = cantidad if (cantidad > 0) else 0
        cantidad = cantidad if (cantidad < len(codGen)) else len(codGen)
        a = np.arange(0,len(codGen)).tolist()
        posMutaciones = np.random.choice(a, size=(cantidad), replace= False).tolist()
        for i in posMutaciones:
            codGen[i] = 0 if (codGen[i] == 1) else 1
        return codGen
    def reemplazoGeneracional(self, padres, hijos):
        return hijos
    def reemplazoElitista(self, padres, hijos):
        poblacion = []
        padrestemp = padres.copy()
        hijostemp = hijos.copy()
        for i in range(0,len(padrestemp)):
            población.append(padrestemp[i])
            población.append(hijostemp[i])
        poblacion.sort(key=lambda x: x.f, reverse=True)

In [66]:
n = 10
bits = 16
iteraciones = 100
sigma = 0.4 #el radio para la funcion de fitness
ag = AlgoritmoGenetico()
funcion = fSenoSharing()
poblacion = ag.iniciarPoblacion(n, bits, funcion,sigma)
for i in range(0, iteraciones):
    padres = ag.selectorTorneo(poblacion)
    hijos = ag.cruceYMutacionGaussianaPoblacion(padres, 0.8, 1,1, funcion,sigma)
    poblacion = ag.reemplazoGeneracional(padres, hijos)
for individuo in poblacion:
    print(individuo.x)
    print(funcion.crecimiento(individuo.x))
    print(individuo.f)
    print("\n")

[1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[8.40477661062467]
0.10651359107104452


[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0]
[8.467094580132525]
0.10223259297468058


[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[8.4057353486171]
0.10645082091335227


[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0]
[8.467094580132525]
0.10223259297468058


[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[8.4057353486171]
0.10645082091335227


[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[8.390395540738243]
0.10744336274069696


[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0]
[9.448842284379335]
-0.024062001102309355


[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[8.4057353486171]
0.10645082091335227


[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[8.390395540738243]
0.10744336274069696


[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0]
[8.957009694263501]
0.4508954163896673




In [4]:
p =[]
n=10
d=1
f= fSeno()
iteraciones=10
dcrowd = deterministic_crowding()
a=0
b=2*math.pi*4
for i in range(0,n):
    temp = individuo()
    x = []
    for j in range(0,d):
        x.append(random.uniform(a,b))
    temp.x = x
    temp.f = f.fitness(temp.x)
    p.append(temp)
for i in range(0,iteraciones):
    ng = dcrowd.procedimietoGeneral(p,f)
for i in range(0,len(ng)):
    print(ng[i].x)
    print(ng[i].fitness)
    print("\n")
    
xfunc = []
yfunc = []


[20.536967417286398]
0.9932081533369308


[22.706243198292448]
-0.6556888630735036


[8.130162348202367]
0.9621039075190194


[14.183736192401954]
0.9989158483732191


[24.287448612296924]
-0.748165298822618


[20.657503909212334]
0.9720110913794308


[8.130162348202367]
0.9621039075190194


[14.183736192401954]
0.9989158483732191


[8.130162348202367]
0.9621039075190194


[7.912489808052147]
0.9982888849916001




In [64]:
column_1 = Column([0.5,1.2], 'x')
column_2 = Column([0.5,1.6], 'y')
column_3 = Column([1.5,1], 'x2')
column_4 = Column([1.5,2], 'y2')
column_5 = Column([0.7,1], 'a')
column_6 = Column([0.7,1], 'b')

grid = Grid([column_1, column_2, column_3, column_4, column_5, column_6])
py.grid_ops.upload(grid, 'ping_pong_grid'+str(time.time()), auto_open=False)

'https://plot.ly/~dabelloa/12/'

In [77]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

updatemenus_dict = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

figure = {
    'data': [],
    'layout': {'title': 'Deterministic crowding',
               'xaxis': {'range': [0, 2], 'autorange': True},
               'yaxis': {'range': [0, 2], 'autorange': True},
              },
    'frames': []
}
frame = {
    'data': [
        {
            'xsrc': grid.get_column_reference('x'),
            'ysrc': grid.get_column_reference('y'),
            'mode': 'markers',
        }
    ]
}
figure['layout']['updatemenus'] = updatemenus_dict
figure['frames'].append(frame)

py.icreate_animations(figure, 'ping_pong'+str(time.time()))

/home/dabelloa/.local/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

